In [ ]:
# 1. Setup Kaggle Environment and Install Libraries
# --------------------------------------------------

# Install ultralytics (for YOLOv8 detection)
!pip install -q ultralytics

# Install Segment Anything
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

# Download SAM model weights (ViT-H)
!wget -q 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'

# Clear output for cleaner notebook
from IPython.display import clear_output
clear_output()

print("Environment setup complete!")

In [ ]:
# 2. Import Necessary Libraries
# -----------------------------
import torch
import cv2
import numpy as np
import os
from tqdm.notebook import tqdm # For progress bars
from PIL import Image
from glob import glob
import shutil # For copying files

# SAM imports
from segment_anything import sam_model_registry, SamPredictor # Use SamPredictor for bounding box prompts

# YOLOv8 imports
from ultralytics import YOLO

In [ ]:
# 3. Define Constants and Paths
# -----------------------------
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

SAM_CHECKPOINT_PATH = 'sam_vit_h_4b8939.pth'
SAM_MODEL_TYPE = "vit_h"

# FOOD101_PATH = '/kaggle/input/food41/food41/'
FOOD101_PATH = 'datasets\food_101' # Adjust if your Food-101 path is different

images_dir = os.path.join(FOOD101_PATH, 'images')
meta_dir = os.path.join(FOOD101_PATH, 'meta') # Contains classes.txt, train.txt, test.txt

# Path to your trained YOLOv8 Object Detection model for Food-101
# You need to have trained this model previously on Food-101 for object detection.
# Example: If you trained it on Food-101 and saved the best weights:
# If you uploaded it as a Kaggle dataset:

# YOLOV8_DETECTION_MODEL_PATH = '/kaggle/input/your-yolov8-food-detection-dataset/best.pt' # ADJUST THIS TO BEST MODEL
# If you just want to test with a general YOLOv8 model (less accurate for food types):
YOLOV8_DETECTION_MODEL_PATH = 'yolov8s.pt' # Downloaded by ultralytics automatically

OUTPUT_LABELS_DIR = 'yolo_sam_autogenerated_seg_labels'
os.makedirs(os.path.join(OUTPUT_LABELS_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_LABELS_DIR, 'labels'), exist_ok=True)

In [ ]:
# 4. Load Food-101 Categories and Create Mappings
# ------------------------------------------------
classes_file = os.path.join(meta_dir, 'classes.txt')
if os.path.exists(classes_file):
    with open(classes_file, 'r') as f:
        categories = [line.strip() for line in f.readlines()]
else:
    categories = sorted(os.listdir(images_dir))
    categories = [d for d in categories if os.path.isdir(os.path.join(images_dir, d))]

num_classes_food101 = len(categories)
category_to_id = {name: i for i, name in enumerate(categories)}
id_to_category = {i: name for i, name in enumerate(categories)}

print(f"Loaded {num_classes_food101} food categories.")

In [ ]:
# 5. Load YOLOv8 Object Detection Model
# -------------------------------------
model_yolo_detector = YOLO(YOLOV8_DETECTION_MODEL_PATH)
model_yolo_detector.to(DEVICE)
print(f"YOLOv8 Detection model loaded from {YOLOV8_DETECTION_MODEL_PATH}")

In [ ]:
# 6. Initialize SAM Predictor
# ---------------------------
sam = sam_model_registry[SAM_MODEL_TYPE](checkpoint=SAM_CHECKPOINT_PATH)
sam.to(device=DEVICE)
sam_predictor = SamPredictor(sam) # Use SamPredictor for specific prompts (bounding boxes)
print("SAM Predictor initialized.")

# Helper function to convert mask to polygon points for YOLOv8 segmentation format
def mask_to_yolov8_polygon(mask, image_width, image_height):
    """Converts a boolean mask to normalized polygon coordinates."""
    # Find contours from the mask
    # `np.ascontiguousarray` is important for cv2.findContours
    binary_mask_uint8 = np.ascontiguousarray(mask.astype(np.uint8) * 255)
    contours, _ = cv2.findContours(binary_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return []

    # Take the largest contour (assuming it's the primary object)
    largest_contour = max(contours, key=cv2.contourArea)

    # Flatten and normalize polygon points
    polygon_points = largest_contour.flatten().tolist()
    
    normalized_polygon = []
    for i in range(0, len(polygon_points), 2):
        x_coord, y_coord = polygon_points[i], polygon_points[i+1]
        normalized_polygon.append(f"{x_coord / image_width:.6f}")
        normalized_polygon.append(f"{y_coord / image_height:.6f}")

In [ ]:
# 7. Automated Detection and Segmentation Loop
# --------------------------------------------

# Get all image paths from Food-101 (or a subset for testing)
all_image_paths = []
for category_name in categories:
    category_path = os.path.join(images_dir, category_name)
    all_image_paths.extend(glob(os.path.join(category_path, '*.jpg')))

# Optional: Process a smaller subset for initial testing
# all_image_paths = all_image_paths[:100]

print(f"Starting automated detection and segmentation for {len(all_image_paths)} images...")

# Define a confidence threshold for YOLOv8 detections
YOLO_CONF_THRESHOLD = 0.5 # Adjust as needed

for image_path in tqdm(all_image_paths, desc="Processing images"):
    try:
        image_bgr = cv2.imread(image_path)
        if image_bgr is None:
            print(f"Warning: Could not read image {image_path}. Skipping.")
            continue
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        h, w, _ = image_bgr.shape

        # Stage 1: YOLOv8 Object Detection
        # Pass the image to YOLOv8 detector
        results_yolo = model_yolo_detector(image_rgb, conf=YOLO_CONF_THRESHOLD, verbose=False) # verbose=False for cleaner output

        yolov8_labels_lines = []

        # Process detections from YOLOv8
        if results_yolo and len(results_yolo[0].boxes) > 0:
            for box_data in results_yolo[0].boxes:
                # Extract bounding box (xyxy format: x_min, y_min, x_max, y_max)
                # Ensure it's on CPU and converted to numpy for SAMPredictor
                bbox_xyxy = box_data.xyxy.cpu().numpy().astype(int).flatten()
                
                # Extract class ID and confidence
                class_id = int(box_data.cls.cpu().item())
                confidence = float(box_data.conf.cpu().item())

                # Make sure the bounding box is valid
                if not len(bbox_xyxy) == 4:
                    continue
                
                # SAM expects bounding boxes in (x_min, y_min, x_max, y_max) format
                input_box_for_sam = np.array(bbox_xyxy)

                # Stage 2: SAM Segmentation using YOLO's Bounding Box as Prompt
                sam_predictor.set_image(image_rgb)
                
                # Use multimask_output=False if you expect one main object per box
                # If a box might contain multiple subtle objects, you could try True and pick best score
                masks, scores, logits = sam_predictor.predict(
                    point_coords=None,
                    point_labels=None,
                    box=input_box_for_sam[None, :], # SAM expects (N, 4) for boxes
                    multimask_output=False
                )
                
                # Take the first (and usually only) mask if multimask_output=False
                if masks.shape[0] > 0:
                    best_mask = masks[0] # This is a boolean mask (H, W)

                    # Convert SAM mask to YOLOv8 polygon format
                    polygon_coords = mask_to_yolov8_polygon(best_mask, w, h)
                    
                    if polygon_coords: # Ensure a valid polygon was extracted
                        # YOLOv8 format: <class_id> <normalized_polygon_coords>
                        yolov8_labels_lines.append(f"{class_id} {' '.join(polygon_coords)}")
        
        # Save the YOLOv8 format labels to a .txt file
        image_filename_base = os.path.splitext(os.path.basename(image_path))[0]
        output_label_path = os.path.join(OUTPUT_LABELS_DIR, 'labels', f"{image_filename_base}.txt")
        
        with open(output_label_path, 'w') as f:
            for line in yolov8_labels_lines:
                f.write(line + '\n')

        # Copy the original image to the output folder to create a complete dataset
        output_image_path = os.path.join(OUTPUT_LABELS_DIR, 'images', os.path.basename(image_path))
        shutil.copyfile(image_path, output_image_path)

    except Exception as e:
        print(f"Error processing {image_path}: {e}")

print("Automated detection and segmentation with YOLO+SAM complete! Labels saved to:", OUTPUT_LABELS_DIR)

In [ ]:
# 8. Prepare for YOLOv8 Instance Segmentation Training (Dataset Split & YAML)
# -------------------------------------------------------------------------
# This section is identical to the previous answer, as the output format is the same.
# You'll use the 'yolo_sam_autogenerated_seg_labels' directory as your source for images and labels.

yolov8_data_root = 'food101_yolov8_seg_dataset_from_yolo_sam'
os.makedirs(os.path.join(yolov8_data_root, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(yolov8_data_root, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(yolov8_data_root, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(yolov8_data_root, 'val', 'labels'), exist_ok=True)
os.makedirs(os.path.join(yolov8_data_root, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(yolov8_data_root, 'test', 'labels'), exist_ok=True)

# Load train.txt and test.txt from Food-101 meta folder
train_image_rel_paths = []
test_image_rel_paths = []

with open(os.path.join(meta_dir, 'train.txt'), 'r') as f:
    train_image_rel_paths = [line.strip() + '.jpg' for line in f.readlines()]
with open(os.path.join(meta_dir, 'test.txt'), 'r') as f:
    test_image_rel_paths = [line.strip() + '.jpg' for line in f.readlines()]

from sklearn.model_selection import train_test_split
val_split_ratio = 0.15 # 15% of training data for validation
train_images_for_yolo, val_images_for_yolo = train_test_split(
    train_image_rel_paths, test_size=val_split_ratio, random_state=42
)

# Function to copy images and their corresponding labels
def copy_files(image_rel_paths, target_image_dir, target_label_dir, source_images_dir, source_labels_dir, original_food101_images_base):
    for rel_path in tqdm(image_rel_paths, desc=f"Copying to {os.path.basename(target_image_dir)}"):
        img_name = os.path.basename(rel_path)
        label_name = os.path.splitext(img_name)[0] + '.txt'

        # Construct path to original Food-101 image (for copying the image)
        src_original_img_path = os.path.join(original_food101_images_base, rel_path)
        # Construct path to the auto-generated label (from our YOLO+SAM process)
        src_auto_label_path = os.path.join(source_labels_dir, label_name)

        dest_img_path = os.path.join(target_image_dir, img_name)
        dest_label_path = os.path.join(target_label_dir, label_name)

        if os.path.exists(src_original_img_path) and os.path.exists(src_auto_label_path):
            shutil.copyfile(src_original_img_path, dest_img_path)
            shutil.copyfile(src_auto_label_path, dest_label_path)
        else:
            # This can happen if an image had no food items detected or below confidence
            # You might choose to skip these or generate empty label files.
            pass # print(f"Warning: Missing file for {img_name} or its label. Skipping copy.")

print("\nCopying files to YOLOv8 dataset structure...")

copy_files(train_images_for_yolo, os.path.join(yolov8_data_root, 'train', 'images'), os.path.join(yolov8_data_root, 'train', 'labels'), images_dir, os.path.join(OUTPUT_LABELS_DIR, 'labels'), images_dir)
copy_files(val_images_for_yolo, os.path.join(yolov8_data_root, 'val', 'images'), os.path.join(yolov8_data_root, 'val', 'labels'), images_dir, os.path.join(OUTPUT_LABELS_DIR, 'labels'), images_dir)
copy_files(test_image_rel_paths, os.path.join(yolov8_data_root, 'test', 'images'), os.path.join(yolov8_data_root, 'test', 'labels'), images_dir, os.path.join(OUTPUT_LABELS_DIR, 'labels'), images_dir)

print("Dataset structure created and files copied.")

# Create data.yaml for YOLOv8 Instance Segmentation training
data_yaml_content = f"""
path: {yolov8_data_root}
train: train/images
val: val/images
test: test/images

nc: {num_classes_food101}
names: {categories}
"""

with open(os.path.join(yolov8_data_root, 'food101_yolov8_seg_yolo_sam.yaml'), 'w') as f:
    f.write(data_yaml_content)

print(f"YOLOv8 data.yaml created at {os.path.join(yolov8_data_root, 'food101_yolov8_seg_yolo_sam.yaml')}")

In [ ]:
# 9. Train YOLOv8 Instance Segmentation Model (on the newly generated dataset)
# ---------------------------------------------------------------------------
from ultralytics import YOLO

# Load a pre-trained YOLOv8 segmentation model
model_yolov8_segmentation = YOLO('yolov8s-seg.pt') # 'n', 'm', 'l', 'x' also available

# Train the model
print("\nStarting YOLOv8 instance segmentation training on YOLO+SAM generated data...")
results = model_yolov8_segmentation.train(
    data=os.path.join(yolov8_data_root, 'food101_yolov8_seg_yolo_sam.yaml'),
    epochs=50, # Adjust
    imgsz=640,
    batch=16,  # Adjust based on GPU memory.
    name='food101_yolo_sam_seg_run',
    device=0
)

print("YOLOv8 instance segmentation training complete!")

In [ ]:
# 10. (Optional) Evaluate and Predict with the final YOLOv8-seg model
# --------------------------------------------------------------------
print("\nEvaluating and predicting with the trained YOLOv8 segmentation model...")
best_model_path_yolov8_seg = os.path.join(model_yolov8_segmentation.trainer.save_dir, 'weights', 'best.pt')
print(f"Best YOLOv8 segmentation model saved at: {best_model_path_yolov8_seg}")

trained_yolov8_seg_model = YOLO(best_model_path_yolov8_seg)

# Evaluate on the test set
metrics = trained_yolov8_seg_model.val()

# Run inference on a sample image from the test set
test_images_folder_for_seg = os.path.join(yolov8_data_root, 'test', 'images')
if os.path.exists(test_images_folder_for_seg) and os.listdir(test_images_folder_for_seg):
    sample_test_image_path_for_seg = os.path.join(test_images_folder_for_seg, os.listdir(test_images_folder_for_seg)[0])
    print(f"Running inference on: {sample_test_image_path_for_seg}")
    predict_results_seg = trained_yolov8_seg_model.predict(source=sample_test_image_path_for_seg, save=True, show_conf=True, show_labels=True)
    
    # Display the predicted image
    from PIL import Image
    from IPython.display import display

    predicted_image_output_dir_seg = predict_results_seg[0].save_dir
    predicted_image_filename_seg = os.path.basename(sample_test_image_path_for_seg)
    if os.path.exists(os.path.join(predicted_image_output_dir_seg, predicted_image_filename_seg)):
        print(f"Prediction result saved to: {os.path.join(predicted_image_output_dir_seg, predicted_image_filename_seg)}")
        display(Image.open(os.path.join(predicted_image_output_dir_seg, predicted_image_filename_seg)))
    else:
        print("Predicted image file not found. Check YOLOv8 segmentation output directory.")
else:
    print("No images found in the test set for inference demonstration.")